In [9]:
# Below is the code to run the files locally after importing. However, the files are temporary and are deleted when the runtime disconnects or restarts.

# con = sqlite3.connect("cademycode_updated.db")
# cursor = con.cursor()
# cursor.execute("SELECT name FROM sqlite_master;")
# print(cursor.fetchall())

[]


In [71]:
# This code is accessing the files from Google Drive, where the database is kept permanently

from google.colab import drive
drive.mount('/content/drive')

con = sqlite3.connect("/content/drive/MyDrive/cademycode_updated.db")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [72]:
import sqlite3
import pandas as pd

# Retrieving the names of the tables in the dataset
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", con)
tables

,name
0,cademycode_students
1,cademycode_courses
2,cademycode_student_jobs


In [73]:
# Reading each table in as a dataframe
students = pd.read_sql_query("SELECT * FROM cademycode_students;", con)
courses = pd.read_sql_query("SELECT * FROM cademycode_courses;", con)
jobs = pd.read_sql_query("SELECT * FROM cademycode_student_jobs;", con)

In [74]:
#Getting familiar with the tables by returning the first 5 rows of each
print("Students:")
display(students.head())

print("Courses:")
display(courses.head())

print("Jobs:")
display(jobs.head())

Students:


,uuid,name,dob,sex,contact_info,job_id,num_course_taken,current_career_path_id,time_spent_hrs
0,1,Annabelle Avery,1943-07-03,F,"{""mailing_address"": ""303 N Timber Key, Irondal...",7.0,6.0,1.0,4.99
1,2,Micah Rubio,1991-02-07,M,"{""mailing_address"": ""767 Crescent Fair, Shoals...",7.0,5.0,8.0,4.4
2,3,Hosea Dale,1989-12-07,M,"{""mailing_address"": ""P.O. Box 41269, St. Bonav...",7.0,8.0,8.0,6.74
3,4,Mariann Kirk,1988-07-31,F,"{""mailing_address"": ""517 SE Wintergreen Isle, ...",6.0,7.0,9.0,12.31
4,5,Lucio Alexander,1963-08-31,M,"{""mailing_address"": ""18 Cinder Cliff, Doyles b...",7.0,14.0,3.0,5.64


Courses:


,career_path_id,career_path_name,hours_to_complete
0,1,data scientist,20
1,2,data engineer,20
2,3,data analyst,12
3,4,software engineering,25
4,5,backend engineer,18


Jobs:


,job_id,job_category,avg_salary
0,1,analytics,86000
1,2,engineer,101000
2,3,software developer,110000
3,4,creative,66000
4,5,financial services,135000


In [75]:
# Custom function to gain info on the students table

def inspect_df(students):
    print("=== DataFrame Info ===")
    students.info()
    print("\n=== Summary Stats (Numeric Columns) ===")
    display(students.describe(include=['int64', 'float64']))
    print("\n=== Unique Values ===")
    for col in students.columns:
        print(f"{col}: {students[col].nunique()}")
    print("\n=== Missing Values ===")
    print(students.isnull().sum())

inspect_df(students)


=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    6000 non-null   int64 
 1   name                    6000 non-null   object
 2   dob                     6000 non-null   object
 3   sex                     6000 non-null   object
 4   contact_info            6000 non-null   object
 5   job_id                  5986 non-null   object
 6   num_course_taken        5733 non-null   object
 7   current_career_path_id  5513 non-null   object
 8   time_spent_hrs          5513 non-null   object
dtypes: int64(1), object(8)
memory usage: 422.0+ KB

=== Summary Stats (Numeric Columns) ===


,uuid
count,6000.000000
mean,3000.500000
std,1732.195139
min,1.000000
25%,1500.750000
50%,3000.500000
75%,4500.250000
max,6000.000000



=== Unique Values ===
uuid: 6000
name: 5998
dob: 5277
sex: 3
contact_info: 6000
job_id: 10
num_course_taken: 16
current_career_path_id: 10
time_spent_hrs: 2448

=== Missing Values ===
uuid                        0
name                        0
dob                         0
sex                         0
contact_info                0
job_id                     14
num_course_taken          267
current_career_path_id    487
time_spent_hrs            487
dtype: int64


In [98]:
# Drop rows with missing values and make a copy
students = students.dropna(subset=['job_id', 'num_course_taken', 'current_career_path_id', 'time_spent_hrs']).copy()

# Converting Date of Birth from a string to a datetime datatype
students['dob'] = pd.to_datetime(students['dob'], errors='coerce')

# Converting Number of Courses Taken, Time Spent Hours, Current Career Path Id, and Job Id from objects to numeric
students['num_course_taken'] = pd.to_numeric(students['num_course_taken'], errors='coerce')
students['time_spent_hrs'] = pd.to_numeric(students['time_spent_hrs'], errors='coerce')
students['current_career_path_id'] = pd.to_numeric(
    students['current_career_path_id'], errors='coerce'
).astype('int64')
students['job_id'] = pd.to_numeric(
    students['job_id'], errors='coerce'
).astype('int64')


In [99]:
# Check for duplicates by all columns
duplicate_count = students.duplicated().sum()
print(f"Total duplicate rows: {duplicate_count}")

# Remove duplicates if any
students = students.drop_duplicates().reset_index(drop=True)

Total duplicate rows: 0


In [100]:
import ast

# Convert string dictionaries to Python dicts and extract mailing address
def extract_address(info):
    try:
        return ast.literal_eval(info).get("mailing_address", None)
    except:
        return None

students['mailing_address'] = students['contact_info'].apply(extract_address)

In [101]:
# Making all column names lowercase and replacing spaces with underscores
students.columns = students.columns.str.strip().str.lower().str.replace(' ', '_')

In [102]:
print(students['sex'].unique())

['F' 'M' nan]


In [103]:
# Null values in sex as 'N.' Replacing 'N' with null
import numpy as np

students['sex'] = students['sex'].replace('N', np.nan)
print(students['sex'].unique())

null_count = students['sex'].isnull().sum()
print(f"Number of null values in 'sex': {null_count}")

['F' 'M' nan]
Number of null values in 'sex': 1085


In [ ]:
# Too many null sex values (~18% of dataset). Keeping them in as null

In [104]:
#Inspecting the table again after all cleaning changes
inspect_df(students)
students[['contact_info', 'mailing_address']].head()

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5254 entries, 0 to 5253
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5254 non-null   int64         
 1   name                    5254 non-null   object        
 2   dob                     5254 non-null   datetime64[ns]
 3   sex                     4169 non-null   object        
 4   contact_info            5254 non-null   object        
 5   job_id                  5254 non-null   int64         
 6   num_course_taken        5254 non-null   float64       
 7   current_career_path_id  5254 non-null   int64         
 8   time_spent_hrs          5254 non-null   float64       
 9   mailing_address         5254 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 410.6+ KB

=== Summary Stats (Numeric Columns) ===


,uuid,job_id,num_course_taken,current_career_path_id,time_spent_hrs
count,5254.000000,5254.000000,5254.000000,5254.000000,5254.000000
mean,3045.984583,4.189380,7.491816,5.485915,12.294970
std,1751.055959,2.303234,4.589428,2.878346,8.025775
min,1.000000,0.000000,0.000000,1.000000,0.000000
25%,1522.250000,2.000000,4.000000,3.000000,5.690000
50%,3064.000000,4.000000,7.000000,5.000000,11.420000
75%,4590.750000,6.000000,12.000000,8.000000,17.870000
max,6000.000000,9.000000,15.000000,10.000000,35.980000



=== Unique Values ===
uuid: 5254
name: 5253
dob: 4692
sex: 2
contact_info: 5254
job_id: 10
num_course_taken: 16
current_career_path_id: 10
time_spent_hrs: 2406
mailing_address: 5254

=== Missing Values ===
uuid                         0
name                         0
dob                          0
sex                       1085
contact_info                 0
job_id                       0
num_course_taken             0
current_career_path_id       0
time_spent_hrs               0
mailing_address              0
dtype: int64


,contact_info,mailing_address
0,"{""mailing_address"": ""303 N Timber Key, Irondal...","303 N Timber Key, Irondale, Wisconsin, 84736"
1,"{""mailing_address"": ""767 Crescent Fair, Shoals...","767 Crescent Fair, Shoals, Indiana, 37439"
2,"{""mailing_address"": ""P.O. Box 41269, St. Bonav...","P.O. Box 41269, St. Bonaventure, Virginia, 83637"
3,"{""mailing_address"": ""517 SE Wintergreen Isle, ...","517 SE Wintergreen Isle, Lane, Arkansas, 82242"
4,"{""mailing_address"": ""18 Cinder Cliff, Doyles b...","18 Cinder Cliff, Doyles borough, Rhode Island,..."


In [105]:
# Using custom function to gain info on the courses table
inspect_df(courses)

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     10 non-null     int64 
 1   career_path_name   10 non-null     object
 2   hours_to_complete  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes

=== Summary Stats (Numeric Columns) ===


,career_path_id,hours_to_complete
count,10.00000,10.000000
mean,5.50000,21.900000
std,3.02765,6.707376
min,1.00000,12.000000
25%,3.25000,18.500000
50%,5.50000,20.000000
75%,7.75000,26.500000
max,10.00000,35.000000



=== Unique Values ===
career_path_id: 10
career_path_name: 10
hours_to_complete: 7

=== Missing Values ===
career_path_id       0
career_path_name     0
hours_to_complete    0
dtype: int64


In [106]:
# Check courses for duplicates
print(f"Duplicate rows: {courses.duplicated().sum()}")

# Check for unique 1 to 1 mapping
print(courses.groupby('career_path_id')['career_path_name'].nunique())

Duplicate rows: 0
career_path_id
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
Name: career_path_name, dtype: int64


In [107]:
# Standardize column names
courses.columns = courses.columns.str.strip().str.lower().str.replace(' ', '_')

# Double check table again. All looks good.
inspect_df(courses)

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     10 non-null     int64 
 1   career_path_name   10 non-null     object
 2   hours_to_complete  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes

=== Summary Stats (Numeric Columns) ===


,career_path_id,hours_to_complete
count,10.00000,10.000000
mean,5.50000,21.900000
std,3.02765,6.707376
min,1.00000,12.000000
25%,3.25000,18.500000
50%,5.50000,20.000000
75%,7.75000,26.500000
max,10.00000,35.000000



=== Unique Values ===
career_path_id: 10
career_path_name: 10
hours_to_complete: 7

=== Missing Values ===
career_path_id       0
career_path_name     0
hours_to_complete    0
dtype: int64


In [108]:
# Inspect final dataframe, jobs
inspect_df(jobs)

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_id        10 non-null     int64 
 1   job_category  10 non-null     object
 2   avg_salary    10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes

=== Summary Stats (Numeric Columns) ===


,job_id,avg_salary
count,10.00000,10.000000
mean,4.50000,84900.000000
std,3.02765,35306.436303
min,0.00000,10000.000000
25%,2.25000,69500.000000
50%,4.50000,83000.000000
75%,6.75000,107750.000000
max,9.00000,135000.000000



=== Unique Values ===
job_id: 10
job_category: 10
avg_salary: 9

=== Missing Values ===
job_id          0
job_category    0
avg_salary      0
dtype: int64


In [109]:
# Check jobs for duplicates
duplicate_jobs = jobs.duplicated().sum()
print(f"Duplicate rows: {duplicate_jobs}")

Duplicate rows: 0


In [110]:
# 3 duplicates found. Printing them
duplicates = jobs[jobs.duplicated(keep=False)]
print(duplicates)

Empty DataFrame
Columns: [job_id, job_category, avg_salary]
Index: []


In [112]:
# Dropping duplicates and reinspecting the dataframe
jobs = jobs.drop_duplicates().reset_index(drop=True)

inspect_df(jobs)

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_id        10 non-null     int64 
 1   job_category  10 non-null     object
 2   avg_salary    10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 372.0+ bytes

=== Summary Stats (Numeric Columns) ===


,job_id,avg_salary
count,10.00000,10.000000
mean,4.50000,84900.000000
std,3.02765,35306.436303
min,0.00000,10000.000000
25%,2.25000,69500.000000
50%,4.50000,83000.000000
75%,6.75000,107750.000000
max,9.00000,135000.000000



=== Unique Values ===
job_id: 10
job_category: 10
avg_salary: 9

=== Missing Values ===
job_id          0
job_category    0
avg_salary      0
dtype: int64


In [114]:
# Merging all dataframes into a single dataframe for export\
master_df = (
    students
    .merge(courses, left_on="current_career_path_id", right_on="career_path_id", how="left")
    .merge(jobs, on="job_id", how="left")
)

master_df.to_csv("clean_master_dataset.csv", index=False)

In [121]:
# Inspecting master dataframe after merge
inspect_df(master_df)

=== DataFrame Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5254 entries, 0 to 5253
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5254 non-null   int64         
 1   name                    5254 non-null   object        
 2   dob                     5254 non-null   datetime64[ns]
 3   sex                     4169 non-null   object        
 4   contact_info            5254 non-null   object        
 5   job_id                  5254 non-null   int64         
 6   num_course_taken        5254 non-null   float64       
 7   current_career_path_id  5254 non-null   int64         
 8   time_spent_hrs          5254 non-null   float64       
 9   mailing_address         5254 non-null   object        
 10  career_path_id          5254 non-null   int64         
 11  career_path_name        5254 non-null   object        
 12  hours_to_complete       5

,uuid,job_id,num_course_taken,current_career_path_id,time_spent_hrs,career_path_id,hours_to_complete,avg_salary
count,5254.000000,5254.000000,5254.000000,5254.000000,5254.000000,5254.000000,5254.000000,5254.000000
mean,3045.984583,4.189380,7.491816,5.485915,12.294970,5.485915,21.825086,87253.901789
std,1751.055959,2.303234,4.589428,2.878346,8.025775,2.878346,6.361863,29667.974489
min,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,12.000000,10000.000000
25%,1522.250000,2.000000,4.000000,3.000000,5.690000,3.000000,18.000000,66000.000000
50%,3064.000000,4.000000,7.000000,5.000000,11.420000,5.000000,20.000000,86000.000000
75%,4590.750000,6.000000,12.000000,8.000000,17.870000,8.000000,27.000000,110000.000000
max,6000.000000,9.000000,15.000000,10.000000,35.980000,10.000000,35.000000,135000.000000



=== Unique Values ===
uuid: 5254
name: 5253
dob: 4692
sex: 2
contact_info: 5254
job_id: 10
num_course_taken: 16
current_career_path_id: 10
time_spent_hrs: 2406
mailing_address: 5254
career_path_id: 10
career_path_name: 10
hours_to_complete: 7
job_category: 10
avg_salary: 9

=== Missing Values ===
uuid                         0
name                         0
dob                          0
sex                       1085
contact_info                 0
job_id                       0
num_course_taken             0
current_career_path_id       0
time_spent_hrs               0
mailing_address              0
career_path_id               0
career_path_name             0
hours_to_complete            0
job_category                 0
avg_salary                   0
dtype: int64


In [122]:
# Final verification to ensure the export file structure is correct
import pandas as pd
test_df = pd.read_csv("clean_master_dataset.csv")
print(test_df.head())

   uuid             name         dob sex  \
0     1  Annabelle Avery  1943-07-03   F   
1     2      Micah Rubio  1991-02-07   M   
2     3       Hosea Dale  1989-12-07   M   
3     4     Mariann Kirk  1988-07-31   F   
4     5  Lucio Alexander  1963-08-31   M   

                                        contact_info  job_id  \
0  {"mailing_address": "303 N Timber Key, Irondal...       7   
1  {"mailing_address": "767 Crescent Fair, Shoals...       7   
2  {"mailing_address": "P.O. Box 41269, St. Bonav...       7   
3  {"mailing_address": "517 SE Wintergreen Isle, ...       6   
4  {"mailing_address": "18 Cinder Cliff, Doyles b...       7   

   num_course_taken  current_career_path_id  time_spent_hrs  \
0               6.0                       1            4.99   
1               5.0                       8            4.40   
2               8.0                       8            6.74   
3               7.0                       9           12.31   
4              14.0                